我们的第 1 项工作是将真实世界的异构数据编码为浮点数张量，以供神经网络使用。

In [35]:
import csv
import torch
import numpy as np
wine_path = '../data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

这里我们只规定了二维数组的类型（ 32 位浮点数）、用于分隔每行数据的分隔符以及不读取第 1 行（因为它包含列名）。

让我们检查一下是否已经读取了所有数据：

In [36]:
col_list = next(csv.reader(open(wine_path), delimiter=";")) # 获取标题行
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [37]:
# 继续将 NumPy 数组转换为 PyTorch 张量。
wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

至此，我们有了一个浮点数的 torch.Tensor 对象，它包含所有的列，也包括表示质量评分的最后一列

我们可以将分数视为一个连续变量，把它当作一个实数，然后执行回归任务，或者将其视为一个标签，并尝试在分类任务中根据化学特征分析猜测标签。在这 2 种方式中，我们通常会从输入数据的张量中删除分数，并将其保存在单独的张量中，这样我们就可以将分数作为基本事实，而不必将其输入模型中：

In [38]:
data = wineq[:,:-1] # 选择所有行和除最后一列以外的所有列
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [39]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

如果我们想要将 target 张量转换为标签张量，我们有 2 种方法，这取决于我们使用分类数据的策略或目的。一种是简单地将标签视为分数的整数向量：

In [40]:
target = wineq[:, -1].long()
target, target.shape

(tensor([6, 6, 6,  ..., 6, 7, 6]), torch.Size([4898]))

如果目标张量是字符串标签，如葡萄酒的颜色， 那么为每个字符串分配一个整数将允许我们采用相同的方法。

另一种方法是构建分数的一个独热编码（ one-hot encoding），即将 10 个分数分别编码到一个由 10 个元素组成的向量中，除了其中一个元素设置为 1，其他所有元素都设置为 0，每个分数都有一个不同的索引。这样，分数为 1 就可以映射为向量(1,0,0,0,0,0,0,0,0,0)，分数为 5 就可以映射为向量(0,0,0,0,1,0,0,0,0,0)，依此类推。请注意，分数对应非 0 元素的索引这一事实纯属偶然：我们可以打乱赋值，从分类的角度来看不会有任何改变。

我们可以使用 scatter_()方法获得一个独热编码， 该方法将沿着参数提供的索引方向将源张量的值填充到输入张量中。

In [41]:
target_onehot = torch.zeros(target.shape[0], 10)
# 在target_onehot张量的第二个维度上，根据target张量的值作为索引，将1.0放置在相应的位置，从而实现one-hot编码
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

关于 scatter_()参数的介绍如下:
- dim：指定在哪个维度上进行操作。
- index：一个索引张量，它指定在dim维度上，src的值应该被放置在哪里。
- src：一个张量，它包含了要被放置的值。

在上面的例子中，dim被设置为1，表示操作在第二个维度（索引从0开始）上进行。index是target.unsqueeze(1)，这是将target张量的维度增加一维，然后用作索引。src被设置为1.0，表示要放置的值是1.0。

换句话说，前面的调用可以这样解读：对于每一行，取目标标签的索引（在我们的例子中索引与分数一致），并将其用作列索引，将值设置为 1.0。最终得到一个对分类信息进行编码的张量。

scatter_()方法的第 2 个参数—索引张量， 要求与我们散射到的元素的张量具有相同的维度。因为 target_onehot 只有 2 个维度，即 4898×10，我们需要使用 unsqueeze()为 target 添加一个额外的虚拟维度

In [42]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

调用 unsqueeze()会在 4898 个元素的一维张量中添加一个维度，将其变为一个大小为 4898×1的二维张量，并不改变其内容，即不添加额外的元素，只是使用一个额外的索引来访问元素。也就是说，我们通过 target[0]访问 target 的第 1 个元素，通过 target_unsqueezed[0,0]来访问与之对应的扩充张量。

我们可以使用 PyTorch 张量 API 中的函数来处理张量表格数据。让我们首先获得每列的平均值和标准差：

In [43]:
data_mean = torch.mean(data, dim=0)
data.shape, data_mean.shape
# data_mean

(torch.Size([4898, 11]), torch.Size([11]))

In [44]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

在本例中， dim=0 表示沿维度 0 执行规约。此时，我们可以通过减去平均值并除以标准差来对数据进行归一化，这有助于学习过程。

In [45]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized.shape

torch.Size([4898, 11])

确定 target 中哪些行对应的分数小于或等于 3：

In [46]:
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

只有 20 个 bad_indexes 记录项被设置为 True，张量 bad_indexes 与张量 target 具有相同的形状，其值为 True 或False 取决于我们的阈值与原始张量 target 的比较结果。

In [49]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

现在我们可以开始把葡萄酒分为好酒、中等酒和劣质酒 3 类。让我们对每一列使用 mean()函数：

In [50]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) * (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


劣质葡萄酒的二氧化硫的含量似乎更高。我们可以用二氧化硫总量的阈值作为区分好酒和劣质酒的粗略标准。 让我们来看看二氧化硫总量低于我们之前计算的平均值的索引：

In [51]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

这意味着通过阈值预测超过一半的葡萄酒是高质量的，接下来，我们需要得到真正好酒的索引：

In [52]:
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

实际上优质葡萄酒的数量要比通过阈值预测的数量多出约 500 瓶，我们已经有了确凿的证据，证明它并不完美。现在我们需要看看我们的预测与实际排名是否相符。我们将在预测索引和实际好酒索引之间执行逻辑与操作（记住，每个索引只是一个 0 和 1 的数组），通过使用二者的交集来判断我们做得怎么样：

In [53]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches/n_predicted, n_matches/n_actual

(2018, 0.74000733406674, 0.6193984039287906)

我们预测正确大约 2000 瓶葡萄酒，由于我们共预测了 2700 瓶葡萄酒，因此约有 74%的概率可以预测一瓶葡萄酒是好酒，这个模型确实是高质量的。不幸的是，整个数据集中好酒有 3200 瓶，而我们只识别出约 61%